In [59]:
#Convert to .py
!jupytext --to py:percent "C:\Users\ebroh\BetaNYC\School Budgets\School-Budget-and-Allocations\notebooks\streamlitVisuals.ipynb"

[jupytext] Reading C:\Users\ebroh\BetaNYC\School Budgets\School-Budget-and-Allocations\notebooks\streamlitVisuals.ipynb in format ipynb
[jupytext] Writing 'C:\Users\ebroh\BetaNYC\School Budgets\School-Budget-and-Allocations\notebooks\streamlitVisuals.py' in format py:percent (destination file replaced)


In [1]:
import pandas as pd
from re import sub
from decimal import Decimal
import streamlit as st

def moneyToFloat(money):
    value = Decimal(sub(r'[^\d.]', '', money))
    return value

# Total Allocations vs Budget

In [21]:
allocations = pd.read_csv(r"C:\Users\ebroh\BetaNYC\School Budgets\School-Budget-and-Allocations\data\district 5\allocation_district_5.csv")
allocations = allocations.drop(columns = ['Unnamed: 0'])
allocations['amountNum'] = allocations['amount'].apply((lambda x: moneyToFloat(x)))
#allocations

In [22]:
totalAllocations = allocations.groupby(by=['location_code', 'fiscal_year'])['amountNum'].sum().reset_index()
totalAllocations = totalAllocations.rename(columns = {'amountNum': 'total'})
totalAllocations['type'] = 'allocations'
#totalAllocations

In [28]:
budget = pd.read_csv(r"C:\Users\ebroh\BetaNYC\School Budgets\School-Budget-and-Allocations\data\district 5\budget_district_5.csv")
budget = budget.drop(columns = ['Unnamed: 0'])
budget['amountNum'] = budget['amount'].apply((lambda x: moneyToFloat(x)))
budget

,location_code,fiscal_year,budget_category,budget_assignment,num_positions,service_type,amount,amountNum
0,M692,2022,Leadership,HEAD OF SCHOOL,1.0,NaN,"$ 197,026",197026
1,M692,2022,Leadership,AP - ORGANIZATION,1.0,NaN,"$ 148,166",148166
2,M692,2022,Coordinator/Supervisor/Dean,PARENT COORDINATOR,1.0,NaN,"$ 43,968",43968
3,M692,2022,Secretary,School Secretary,2.0,NaN,"$ 149,545",149545
4,M692,2022,High School Departments,ENGLISH LANGUAGE ARTS,1.0,SUBJECT SPECIFIC,"$ 106,797",106797
...,...,...,...,...,...,...,...,...
7677,M030,2018,OTPS,TELECOMMUNICATION CONTRACTS,NaN,NaN,$ 446,446
7678,M030,2018,OTPS,TEXTBOOKS,NaN,NaN,"$ 7,756",7756
7679,M030,2018,OTPS,TRANSPORTATION OF PUPILS - CONTRACTUAL,NaN,NaN,"$ 2,765",2765
7680,M030,2018,OTPS,TRANSPORTATION OF STAFF - NON-CONTRACT,NaN,NaN,"$ 1,412",1412


In [24]:
totalBudget = budget.groupby(by=['location_code', 'fiscal_year'])['amountNum'].sum().reset_index()
totalBudget = totalBudget.rename(columns = {'amountNum': 'total'})
totalBudget['type'] = 'budget'
#totalBudget

In [25]:
totals = pd.concat([totalBudget, totalAllocations])
#totals.sample(15)

In [27]:
totalsPivot = totals.pivot(index = ['location_code', 'type'], columns = ['fiscal_year'], values = ['total']).reset_index()
# Flatten the MultiIndex columns
totalsPivot.columns = totalsPivot.columns.to_flat_index()

# Rename the columns
totalsPivot = totalsPivot.reset_index()
new_columns = ['index', 'location_code', 'type', '2018', '2019', '2020', '2021', '2022']
totalsPivot.columns = new_columns
totalsPivot =  totalsPivot.drop(columns = 'index')
totalsPivot

,location_code,type,2018,2019,2020,2021,2022
0,M030,allocations,5835002,5817134,5736135,5663435,5490561
1,M030,budget,5830440,5702230,5677318,5586738,5490551
2,M036,allocations,6575446,6774786,6908722,6909623,7355254
3,M036,budget,6480667,6705048,6837152,6853434,7262792
4,M046,allocations,7660059,8358052,8247303,8363241,9354042
5,M046,budget,7620097,8325404,8239311,8357774,9320309
6,M092,allocations,4464084,4647008,4666921,4639564,4538439
7,M092,budget,4445337,4630609,4607173,4587448,4522231
8,M123,allocations,8085767,7481821,6992685,6909528,7888131
9,M123,budget,7904112,7457720,6893315,6822129,7805259


# Funding by Category

In [65]:
allocationCategories = pd.read_csv(r"C:\Users\ebroh\BetaNYC\School Budgets\School-Budget-and-Allocations\data\allocation_category_lookup.csv")
allocationCategories

,allocation_category,allocation_category_group
0,Contract for Excellence FY 09,Contract for Excellence
1,Contract for Excellence FY 09 HS,Contract for Excellence
2,TL Fair Student Funding,Fair Student Funding
3,TL Fair Student Funding HS,Fair Student Funding
4,TL Funds Over Formula,Fair Student Funding
...,...,...
726,Self Sustaining 14,Other
727,Self Sustaining 35,Other
728,Self Sustaining 12,Other
729,SLP Leg Grant 03,Other


In [112]:
allocationCategories['allocation_category_group'].unique()

array(['Contract for Excellence', 'Fair Student Funding',
       'TL SE Specialized Programs ASD', 'Other',
       'School Support Supplement', 'Preschool', 'Related Services',
       'Title I', 'Title 1'], dtype=object)

In [66]:
district_5_allocation_categories = allocations.merge(allocationCategories, how = 'left', left_on = 'allocation_category', right_on = 'allocation_category')
district_5_allocation_categories

,allocation_category,amount,location_code,fiscal_year,amountNum,allocation_category_group
0,AIDP Attendance Shared,"$ 150,490",M692,2022,150490,Other
1,ARPA Academic Recovery,"$ 112,682",M692,2022,112682,Other
2,ARPA Academic Recovery Arts,"$ 28,171",M692,2022,28171,Other
3,ARPA Academic Recovery SE After-School Instruc...,"$ 16,242",M692,2022,16242,Other
4,ARPA Academic Recovery SE Counseling,"$ 2,223",M692,2022,2223,Other
...,...,...,...,...,...,...
6901,TL Summer in the City Shared,"$ 12,969",M030,2018,12969,Other
6902,TL Translation Services,"$ 1,027",M030,2018,1027,Other
6903,TL Vision for School Improvement,"$ 19,097",M030,2018,19097,Other
6904,Universal PreK Quality Full-Day,"$ 205,776",M030,2018,205776,Preschool


In [69]:
funding_per_category = district_5_allocation_categories.groupby(by = ['location_code', 'fiscal_year', 'allocation_category_group'])['amountNum'].sum().reset_index()
funding_per_category

,location_code,fiscal_year,allocation_category_group,amountNum
0,M030,2018,Contract for Excellence,109432
1,M030,2018,Fair Student Funding,3228153
2,M030,2018,Other,837965
3,M030,2018,Preschool,634955
4,M030,2018,Related Services,518102
...,...,...,...,...
824,M692,2022,Contract for Excellence,45445
825,M692,2022,Fair Student Funding,4348117
826,M692,2022,Other,978364
827,M692,2022,Related Services,84288


In [75]:
funding_per_category_pivot = funding_per_category.pivot(index = ['location_code', 'allocation_category_group'], columns = ['fiscal_year'], values = 'amountNum').reset_index()
funding_per_category_pivot

fiscal_year,location_code,allocation_category_group,2018,2019,2020,2021,2022
0,M030,Contract for Excellence,109432,106221,104345,104345,104345
1,M030,Fair Student Funding,3228153,2904842,2738880,2487667,2148210
2,M030,Other,837965,942101,724539,865320,1021966
3,M030,Preschool,634955,924557,1132875,1138401,1205471
4,M030,Related Services,518102,460117,512688,509531,457142
...,...,...,...,...,...,...,...
169,M692,Contract for Excellence,52732,48695,45445,45445,45445
170,M692,Fair Student Funding,3847378,3881761,3966819,4052555,4348117
171,M692,Other,436172,736457,679591,576692,978364
172,M692,Related Services,63311,67023,39749,68136,84288


# Enrollment and FSF

In [81]:
enrollment = pd.read_csv(r"C:\Users\ebroh\BetaNYC\School Budgets\School-Budget-and-Allocations\data\district 5\district_5_demographic_data.csv")
enrollment

,Unnamed: 0,DBN,School Name,Year,Total Enrollment,Grade 3K,Grade PK (Half Day & Full Day),Grade K,Grade 1,Grade 2,...,% White,# Missing Race/Ethnicity Data,% Missing Race/Ethnicity Data,# Students with Disabilities,% Students with Disabilities,# English Language Learners,% English Language Learners,# Poverty,% Poverty,Economic Need Index
0,1095,05M030,P.S. 030 Hernandez/Hughes,2017-18,252,0,16,48,38,38,...,0.024,0,0.000,75,0.298,30,0.119,239,94.8%,Above 95%
1,1096,05M030,P.S. 030 Hernandez/Hughes,2018-19,253,15,15,45,39,32,...,0.032,0,0.000,60,0.237,22,0.087,235,92.9%,93.8%
2,1097,05M030,P.S. 030 Hernandez/Hughes,2019-20,236,17,17,29,31,37,...,0.021,0,0.000,63,0.267,16,0.068,Above 95%,Above 95%,93.2%
3,1098,05M030,P.S. 030 Hernandez/Hughes,2020-21,205,13,22,15,28,25,...,0.020,0,0.000,57,0.278,15,0.073,192,93.7%,92.6%
4,1099,05M030,P.S. 030 Hernandez/Hughes,2021-22,171,9,21,16,22,29,...,0.029,0,0.000,52,0.304,18,0.105,161,94.2%,91.2%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,1227,05M692,"High School for Mathematics, Science and Engin...",2017-18,500,0,0,0,0,0,...,0.272,47,0.094,25,0.050,1,0.002,227,45.4%,41.7%
131,1228,05M692,"High School for Mathematics, Science and Engin...",2018-19,491,0,0,0,0,0,...,0.303,57,0.116,27,0.055,1,0.002,204,41.5%,39.3%
132,1229,05M692,"High School for Mathematics, Science and Engin...",2019-20,485,0,0,0,0,0,...,0.291,60,0.124,28,0.058,0,0.000,180,37.1%,37.9%
133,1230,05M692,"High School for Mathematics, Science and Engin...",2020-21,496,0,0,0,0,0,...,0.280,48,0.097,30,0.060,0,0.000,183,36.9%,38.1%


In [82]:
enrollment.columns

Index(['Unnamed: 0', 'DBN', 'School Name', 'Year', 'Total Enrollment',
       'Grade 3K', 'Grade PK (Half Day & Full Day)', 'Grade K', 'Grade 1',
       'Grade 2', 'Grade 3', 'Grade 4', 'Grade 5', 'Grade 6', 'Grade 7',
       'Grade 8', 'Grade 9', 'Grade 10', 'Grade 11', 'Grade 12', '# Female',
       '% Female', '# Male', '% Male', '# Asian', '% Asian', '# Black',
       '% Black', '# Hispanic', '% Hispanic', '# Multi-Racial',
       '% Multi-Racial', '# Native American', '% Native American', '# White',
       '% White', '# Missing Race/Ethnicity Data',
       '% Missing Race/Ethnicity Data', '# Students with Disabilities',
       '% Students with Disabilities', '# English Language Learners',
       '% English Language Learners', '# Poverty', '% Poverty',
       'Economic Need Index'],
      dtype='object')

In [91]:
def removeLeadingZero(string):
    return(sub(r'^\d\d', '', string))

def year2FY(year):
    return(sub(r'-\d\d$', '', year))

In [93]:
enrollment['DBN'] = enrollment['DBN'].apply(lambda x: removeLeadingZero(x))
enrollment['fiscal_year'] = enrollment['Year'].apply(lambda x: year2FY(x))
enrollment

,Unnamed: 0,DBN,School Name,Year,Total Enrollment,Grade 3K,Grade PK (Half Day & Full Day),Grade K,Grade 1,Grade 2,...,# Missing Race/Ethnicity Data,% Missing Race/Ethnicity Data,# Students with Disabilities,% Students with Disabilities,# English Language Learners,% English Language Learners,# Poverty,% Poverty,Economic Need Index,fiscal_year
0,1095,M030,P.S. 030 Hernandez/Hughes,2017-18,252,0,16,48,38,38,...,0,0.000,75,0.298,30,0.119,239,94.8%,Above 95%,2017
1,1096,M030,P.S. 030 Hernandez/Hughes,2018-19,253,15,15,45,39,32,...,0,0.000,60,0.237,22,0.087,235,92.9%,93.8%,2018
2,1097,M030,P.S. 030 Hernandez/Hughes,2019-20,236,17,17,29,31,37,...,0,0.000,63,0.267,16,0.068,Above 95%,Above 95%,93.2%,2019
3,1098,M030,P.S. 030 Hernandez/Hughes,2020-21,205,13,22,15,28,25,...,0,0.000,57,0.278,15,0.073,192,93.7%,92.6%,2020
4,1099,M030,P.S. 030 Hernandez/Hughes,2021-22,171,9,21,16,22,29,...,0,0.000,52,0.304,18,0.105,161,94.2%,91.2%,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,1227,M692,"High School for Mathematics, Science and Engin...",2017-18,500,0,0,0,0,0,...,47,0.094,25,0.050,1,0.002,227,45.4%,41.7%,2017
131,1228,M692,"High School for Mathematics, Science and Engin...",2018-19,491,0,0,0,0,0,...,57,0.116,27,0.055,1,0.002,204,41.5%,39.3%,2018
132,1229,M692,"High School for Mathematics, Science and Engin...",2019-20,485,0,0,0,0,0,...,60,0.124,28,0.058,0,0.000,180,37.1%,37.9%,2019
133,1230,M692,"High School for Mathematics, Science and Engin...",2020-21,496,0,0,0,0,0,...,48,0.097,30,0.060,0,0.000,183,36.9%,38.1%,2020


In [106]:
enrollmentFinal = pd.DataFrame()
enrollmentFinal = enrollment[['DBN','fiscal_year', 'Total Enrollment', 'Grade 3K', 'Grade PK (Half Day & Full Day)']]
enrollmentFinal['3K-PreK_enrollment'] = enrollmentFinal['Grade 3K'] + enrollmentFinal['Grade PK (Half Day & Full Day)']
enrollmentFinal['K-5_enrollment'] = enrollmentFinal['Total Enrollment'] - enrollmentFinal['3K-PreK_enrollment']
enrollmentFinal = enrollmentFinal.drop(columns = ['Grade 3K', 'Grade PK (Half Day & Full Day)'])
enrollmentFinal

C:\Users\ebroh\AppData\Local\Temp\ipykernel_41444\724273399.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  enrollmentFinal['3K-PreK_enrollment'] = enrollmentFinal['Grade 3K'] + enrollmentFinal['Grade PK (Half Day & Full Day)']
C:\Users\ebroh\AppData\Local\Temp\ipykernel_41444\724273399.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  enrollmentFinal['K-5_enrollment'] = enrollmentFinal['Total Enrollment'] - enrollmentFinal['3K-PreK_enrollment']


,DBN,fiscal_year,Total Enrollment,3K-PreK_enrollment,K-5_enrollment
0,M030,2017,252,16,236
1,M030,2018,253,30,223
2,M030,2019,236,34,202
3,M030,2020,205,35,170
4,M030,2021,171,30,141
...,...,...,...,...,...
130,M692,2017,500,0,500
131,M692,2018,491,0,491
132,M692,2019,485,0,485
133,M692,2020,496,0,496


In [136]:
pivot1 = enrollmentFinal.pivot(index = ['DBN'], columns = ['fiscal_year'], values = '3K-PreK_enrollment').reset_index()
pivot1['Grades'] = '3K-PreK'
pivot2 = enrollmentFinal.pivot(index = ['DBN'], columns = ['fiscal_year'], values = 'K-5_enrollment').reset_index()
pivot2['Grades'] = 'K-5'
enrollmentPivot = pd.concat([pivot1, pivot2])
enrollmentPivot = enrollmentPivot.rename(columns = {'Grades':'Type'})
temp = funding_per_category_pivot[funding_per_category_pivot['allocation_category_group'].isin(['Fair Student Funding', 'Preschool'])]
temp = temp.rename(columns = {'allocation_category_group': 'Type', 'location_code': 'DBN', 2018:'2018', 2019:'2019', 2020:'2020', 2021:'2021'})
temp = temp.drop(columns = [2022])
enrollmentPivot = enrollmentPivot.drop(columns = '2017')
enrollmentPivot = pd.concat([enrollmentPivot, temp])
enrollmentPivot

fiscal_year,DBN,2018,2019,2020,2021,Type
0,M030,30,34,35,30,3K-PreK
1,M036,69,92,42,54,3K-PreK
2,M046,51,44,36,51,3K-PreK
3,M092,18,17,10,15,3K-PreK
4,M123,49,60,40,54,3K-PreK
...,...,...,...,...,...,...
154,M514,1408207,1469664,1582416,1554330,Fair Student Funding
159,M517,1699861,2018023,2358500,2642484,Fair Student Funding
161,M517,186653,197712,427523,427565,Preschool
165,M670,3475928,3827873,4150704,4326225,Fair Student Funding


# Staff vs Enrollment

In [148]:
teaching_budget = budget[budget['budget_category'].isin(['Paraprofessionals', 'Classroom Teacher', 'Homeroom Teacher', 'Elementary Cluster/Quota', 'Cluster/Quota Teacher'])]
teaching_budget['budget_category'] = teaching_budget['budget_category'].replace(['Classroom Teacher', 'Homeroom Teacher', 'Elementary Cluster/Quota', 'Cluster/Quota Teacher'], 'Teacher')
teaching_budget.sample(10)

C:\Users\ebroh\AppData\Local\Temp\ipykernel_41444\3274682473.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  teaching_budget['budget_category'] = teaching_budget['budget_category'].replace(['Classroom Teacher', 'Homeroom Teacher', 'Elementary Cluster/Quota', 'Cluster/Quota Teacher'], 'Teacher')


,budget_assignment,num_positions,amount,budget_category,service_type,location_code,fiscal_year,amountNum
6466,Pre K,2.0,"$ 267,811",Teacher,WHOLE CLASS,M123,2022,267811
5255,45,1.0,"$ 90,523",Teacher,SELF-CONTAINED SP ED,M154,2018,90523
5317,SPECIAL EDUCATION (OR ALL SUBJECTS),4.0,"$ 275,461",Teacher,Sub-Total,M148,2022,275461
3536,Pre K,1.0,"$ 129,642",Teacher,WHOLE CLASS,M197,2019,129642
4255,1,2.0,"$ 219,306",Teacher,Sub-Total,M161,2022,219306
6728,3,1.0,"$ 88,421",Teacher,ICT - General Ed Teacher,M123,2019,88421
4541,1,1.0,"$ 88,931",Teacher,WHOLE CLASS,M161,2019,88931
4563,6,2.0,"$ 178,113",Teacher,WHOLE CLASS,M161,2019,178113
3692,5,1.0,"$ 88,485",Teacher,ICT - General Ed Teacher,M194,2022,88485
6725,2,1.0,"$ 83,391",Teacher,ICT - Special Ed Teacher,M123,2019,83391


In [152]:
positions_per_school = teaching_budget.groupby(by = ['location_code', 'fiscal_year', 'budget_category'])['num_positions'].sum().reset_index()
positions_per_school

,location_code,fiscal_year,budget_category,num_positions
0,M030,2018,Paraprofessionals,10.0
1,M030,2018,Teacher,53.0
2,M030,2019,Paraprofessionals,12.0
3,M030,2019,Teacher,50.0
4,M030,2020,Paraprofessionals,12.0
...,...,...,...,...
234,M670,2022,Paraprofessionals,6.0
235,M692,2018,Paraprofessionals,1.0
236,M692,2019,Paraprofessionals,1.0
237,M692,2021,Paraprofessionals,1.0


In [168]:
positions_per_school_pivot = positions_per_school.pivot(index = ['location_code', 'budget_category'], columns = 'fiscal_year', values = 'num_positions').reset_index()
positions_per_school_pivot = positions_per_school_pivot.rename(columns = {'budget_category': 'type'})
positions_per_school_pivot = positions_per_school_pivot.rename(columns = {2018:'2018', 2019:'2019', 2020:'2020', 2021:'2021'})
positions_per_school_pivot

fiscal_year,location_code,type,2018,2019,2020,2021,2022
0,M030,Paraprofessionals,10.0,12.0,12.0,12.0,11.0
1,M030,Teacher,53.0,50.0,51.0,37.0,31.0
2,M036,Paraprofessionals,20.0,16.0,17.0,18.0,20.0
3,M036,Teacher,60.8,55.0,57.0,55.0,43.0
4,M046,Paraprofessionals,10.0,14.0,19.0,21.0,18.0
5,M046,Teacher,85.0,82.0,84.0,76.0,70.0
6,M092,Paraprofessionals,12.0,11.0,11.0,10.0,7.0
7,M092,Teacher,38.0,32.0,33.0,31.0,24.0
8,M123,Paraprofessionals,7.0,7.0,8.0,12.0,7.0
9,M123,Teacher,65.0,62.0,64.0,62.0,62.0


In [170]:
totalEnrollment = enrollment[['DBN','fiscal_year', 'Total Enrollment']]
totalEnrollment = totalEnrollment.rename(columns = {'DBN': 'location_code'})
totalEnrollmentPivot = totalEnrollment.pivot(index = 'location_code', columns = 'fiscal_year', values = 'Total Enrollment').reset_index()
totalEnrollmentPivot['type'] = 'Total Enrollment'
staffAndEnrollment = pd.concat([totalEnrollmentPivot,positions_per_school_pivot])
staffAndEnrollment = staffAndEnrollment.drop(columns = ['2017', 2022])
staffAndEnrollment

fiscal_year,location_code,2018,2019,2020,2021,type
0,M030,253.0,236.0,205.0,171.0,Total Enrollment
1,M036,355.0,374.0,312.0,290.0,Total Enrollment
2,M046,630.0,557.0,490.0,479.0,Total Enrollment
3,M092,259.0,243.0,214.0,210.0,Total Enrollment
4,M123,471.0,461.0,418.0,430.0,Total Enrollment
...,...,...,...,...,...,...
46,M514,20.0,17.0,24.0,28.0,Teacher
47,M517,3.0,4.0,4.0,5.0,Paraprofessionals
48,M517,24.0,26.0,41.0,42.0,Teacher
49,M670,3.0,2.0,4.0,7.0,Paraprofessionals


# Budget Per Staff Member

In [173]:
pos_and_budget = teaching_budget.groupby(by = ['location_code', 'fiscal_year', 'budget_category'])['amountNum'].sum().reset_index()
pos_and_budget

,location_code,fiscal_year,budget_category,amountNum
0,M030,2018,Paraprofessionals,414942
1,M030,2018,Teacher,5107239
2,M030,2019,Paraprofessionals,495617
3,M030,2019,Teacher,4587081
4,M030,2020,Paraprofessionals,543100
...,...,...,...,...
234,M670,2022,Paraprofessionals,154407
235,M692,2018,Paraprofessionals,39317
236,M692,2019,Paraprofessionals,40779
237,M692,2021,Paraprofessionals,39294


In [177]:
budget_per_staff = pos_and_budget.merge(positions_per_school, on = ['location_code','fiscal_year','budget_category'])
budget_per_staff['average_budget'] = budget_per_staff['amountNum'].astype(float)/budget_per_staff['num_positions']
budget_per_staff

,location_code,fiscal_year,budget_category,amountNum,num_positions,average_budget
0,M030,2018,Paraprofessionals,414942,10.0,41494.200000
1,M030,2018,Teacher,5107239,53.0,96363.000000
2,M030,2019,Paraprofessionals,495617,12.0,41301.416667
3,M030,2019,Teacher,4587081,50.0,91741.620000
4,M030,2020,Paraprofessionals,543100,12.0,45258.333333
...,...,...,...,...,...,...
234,M670,2022,Paraprofessionals,154407,6.0,25734.500000
235,M692,2018,Paraprofessionals,39317,1.0,39317.000000
236,M692,2019,Paraprofessionals,40779,1.0,40779.000000
237,M692,2021,Paraprofessionals,39294,1.0,39294.000000
